# Healthcare Blockchain

In [1]:
cd ../unique_blockchain

C:\Users\Edoardo\PycharmProjects\HealthcareBlockchain\unique_blockchain


In [2]:
from blockchain import Blockchain
from agents import Doctor, Miner, Patient, Minister
import nacl.signing

# Initiate Blockchain

When we start the blockchain we pass an object Minister, that the blockchain instantiates immediately. We can change the specific instantiation of Minister depending on the country. With the object Minister we also want to pass a dictionary of medicines that the Minister of health showed that are incompatible with some illnesses. 

In [3]:

incompatibilities = {'medicine1': ['illness1', 'illness2'],
                    'medicine2': ['illness1'],
                    'medicine3': ['illness2', 'illness3']}

Health_block = Blockchain(Minister, incompatibilities)

Now we can access the object Minister via the Blockchain object. We might want, for example, to retrieve the incompatibilites or adding new ones

In [5]:
Health_block.Minister.incompatibilities

{'medicine1': ['illness1', 'illness2'],
 'medicine2': ['illness1'],
 'medicine3': ['illness2', 'illness3']}

In [9]:
Health_block.Minister.incompatibilities['medicine4'] =  ['illness2']

In [10]:
Health_block.Minister.incompatibilities

{'medicine1': ['illness1', 'illness2'],
 'medicine2': ['illness1'],
 'medicine3': ['illness2', 'illness3'],
 'medicine4': ['illness2']}

# Instantiate the agents

When we have the blockchain ready, we can think that the users start opening their account

In [11]:
# Miners
miner1 = Miner(Health_block)
miner2 = Miner(Health_block)
miner3 = Miner(Health_block)
miner4 = Miner(Health_block)

# Doctors
doctor1 = Doctor('Doctor Muller')
doctor2 = Doctor('Doctor Johannes')

# Patients
patient1 = Patient('Mr. Black')
patient2 = Patient('Mr. Green')
patient3 = Patient('Mr. White')

Every type of agent has its unique attributes. For instance:
    1. The Doctor has the authorization which is by default eual to None and it takes a valid value when the Minister sends him the authorization.
    2. The Patient has the history of the illnesses, which are initiated with an empty list.
    3. Each Miner has the complete history of the blockhchain and a wallet in which the fees are charged.

In [14]:
print('---------doctor---------')
print(doctor1.authorization)

print('---------patient---------')
print(patient1.illnesses)

print('---------miner---------')
print(miner1.chain)
print(miner1.wallet)

---------doctor---------
None
---------patient---------
[]
---------miner---------
[{'index': 1, 'timestamp': 1576407454.3041582, 'transactions': [], 'nounce': 100, 'previous_hash': 1}]
0


# Simulations

## No authorization

Now no doctor has an authorization, so let's see what happens if a doctor tries to do some diagnoses with no authorization

In [17]:
# We first create the 2 transactions
Health_block.new_diagnosis(doctor2, patient1, 'illness2', 0.20)
Health_block.new_diagnosis(doctor2, patient3, 'illness3', 0.20)

# Then we mine a block including all the miners
miners = [miner1, miner2, miner3, miner4]
Health_block.mine(miners)

Not valid Doctor authorization
Not valid Doctor authorization
Not valid Doctor authorization
Not valid Doctor authorization
Mined a new block with number 2
Block number 2 broadcasted to all the nodes


What happens is that every miner reject the transaction because the authorization cannot be opened using the public key of the Minister. So the block is mined, but without transaction. Let's inspect it

In [18]:
Health_block.chain

[{'index': 1,
  'timestamp': 1576407454.3041582,
  'transactions': [],
  'nounce': 100,
  'previous_hash': 1},
 {'index': 2,
  'timestamp': 1576408211.6219943,
  'transactions': [],
  'nounce': 226,
  'previous_hash': '4f53cda18c2baa0c0354bb5f9a3ecbe5ed12ab4d8e11ba873c2f11161202b945'}]

We can see that the second block does not have transactions and no illnessess have been appended to the history of the patients

In [22]:
print(patient1.illnesses)
print(patient3.illnesses)

[]
[]


## Authorization

Now let's create a new authorization by the Minister for the doctor1

In [24]:
Health_block.new_authorization(doctor1, 0.50)
Health_block.mine(miners)

Mined a new block with number 3
Block number 3 broadcasted to all the nodes


In [25]:
Health_block.chain

[{'index': 1,
  'timestamp': 1576407454.3041582,
  'transactions': [],
  'nounce': 100,
  'previous_hash': 1},
 {'index': 2,
  'timestamp': 1576408211.6219943,
  'transactions': [],
  'nounce': 226,
  'previous_hash': '4f53cda18c2baa0c0354bb5f9a3ecbe5ed12ab4d8e11ba873c2f11161202b945'},
 {'index': 3,
  'timestamp': 1576408632.2123766,
  'transactions': [{'type': 'authorization',
    'sender': <agents.Minister at 0x17756c06e48>,
    'recipient': <agents.Doctor at 0x17756c2dc88>,
    'authorization': b"'\xf9\xf5\x9a\xdd\xfaZ\x9bC\xa1%\xad\x8f\x17P^\\\xfe\x82\xb0/\x9f\x12\x05V\xad\xf7\xff\x11\xbcK\xfb@\xb7^\x1d\x9f\xe6\xe0?\xf8\x0b\xb84Wb\x8c\xf4\xbb\xc1*\x8f\x99x\xf7\xc6\xcfr)\xe6|IJ\n32708382f6ed7ab0fa61fb00bf7deca056e603149268753ffd28c2b48c678dd8",
    'fee': 0.5}],
  'nounce': 346,
  'previous_hash': '4f53cda18c2baa0c0354bb5f9a3ecbe5ed12ab4d8e11ba873c2f11161202b945'}]

In the transaction of the last block we now can see the authorization issued by the Minister to the doctor. If we inspect the doctor authorization now we should see the unique code shown in the transaction

In [26]:
doctor1.authorization

b"'\xf9\xf5\x9a\xdd\xfaZ\x9bC\xa1%\xad\x8f\x17P^\\\xfe\x82\xb0/\x9f\x12\x05V\xad\xf7\xff\x11\xbcK\xfb@\xb7^\x1d\x9f\xe6\xe0?\xf8\x0b\xb84Wb\x8c\xf4\xbb\xc1*\x8f\x99x\xf7\xc6\xcfr)\xe6|IJ\n32708382f6ed7ab0fa61fb00bf7deca056e603149268753ffd28c2b48c678dd8"

If doctor1 tries now to do diagnoses, these will have immediate effects on patients

In [28]:
Health_block.new_diagnosis(doctor1, patient3, 'illness1', 0.20)
Health_block.mine(miners)

Mined a new block with number 4
Block number 4 broadcasted to all the nodes


In [29]:
patient3.illnesses

['illness1']

## Fake Authorization

Let's see what happens if a doctor, say doctor2, creates a fake authorization and put it in its own variable authorization

In [32]:
# We first create a random privte key
private_key_random = nacl.signing.SigningKey.generate()

# We then sign the address of the doctor with that key
signed = private_key_random.sign(bytes(doctor2.address, 'utf-8'))

# Finally we insert the authorization in the variable
doctor2.authorization = signed


In [33]:
print(doctor2.authorization)

b'\xdcZ\x03\xef\xff\x08\xcf\xaf]sF\t\x8a/\xa0\xab\x07\xdb\xee/L\x9c\xe9\xc2qY\xc2x!L=y\x8d\xb7\xac\x07\xe4n\\\x1e\xbey\xd9\x1c;65\xa6o!b\xa9\x8f\x9a\xec\xa0N9\xd2\xf5\x8c\\\x11\x050f0bc04036b9ec20467e1e8f4c960fda577514b613e1da5ca1684bae1cbf1fbd'


In [34]:
Health_block.new_diagnosis(doctor2, patient1, 'illness2', 0.20)
Health_block.mine(miners)

Not valid Doctor authorization
Mined a new block with number 5
Block number 5 broadcasted to all the nodes


THe miners detect the fake authorization and the last block will not contain not valida transactions

In [36]:
Health_block.chain

[{'index': 1,
  'timestamp': 1576407454.3041582,
  'transactions': [],
  'nounce': 100,
  'previous_hash': 1},
 {'index': 2,
  'timestamp': 1576408211.6219943,
  'transactions': [],
  'nounce': 226,
  'previous_hash': '4f53cda18c2baa0c0354bb5f9a3ecbe5ed12ab4d8e11ba873c2f11161202b945'},
 {'index': 3,
  'timestamp': 1576408632.2123766,
  'transactions': [{'type': 'authorization',
    'sender': <agents.Minister at 0x17756c06e48>,
    'recipient': <agents.Doctor at 0x17756c2dc88>,
    'authorization': b"'\xf9\xf5\x9a\xdd\xfaZ\x9bC\xa1%\xad\x8f\x17P^\\\xfe\x82\xb0/\x9f\x12\x05V\xad\xf7\xff\x11\xbcK\xfb@\xb7^\x1d\x9f\xe6\xe0?\xf8\x0b\xb84Wb\x8c\xf4\xbb\xc1*\x8f\x99x\xf7\xc6\xcfr)\xe6|IJ\n32708382f6ed7ab0fa61fb00bf7deca056e603149268753ffd28c2b48c678dd8",
    'fee': 0.5}],
  'nounce': 346,
  'previous_hash': '4f53cda18c2baa0c0354bb5f9a3ecbe5ed12ab4d8e11ba873c2f11161202b945'},
 {'index': 4,
  'timestamp': 1576408785.7469268,
  'transactions': [{'type': 'diagnosis',
    'sender': <agents.Doctor 

# Prescription

Another feature of the blockchain is that prevents doctors to do prescriptions of medicines not compatible with the illness history of the patient

In [37]:
Health_block.Minister.incompatibilities

{'medicine1': ['illness1', 'illness2'],
 'medicine2': ['illness1'],
 'medicine3': ['illness2', 'illness3'],
 'medicine4': ['illness2']}

In [38]:
patient3.illnesses

['illness1']

if we see above, patient3 had illness1 and looking at the incompatibilites declared by the minister medicine1 and medicine2 are not compatible with this illness

In [39]:
# Let's try to do a prescription incompatible with the illnesses of patient3
Health_block.new_prescription(doctor1, patient3, 'medicine2', 0.20)
Health_block.mine(miners)

Incompatibility of one prescription with the history of the patient
Mined a new block with number 6
Block number 6 broadcasted to all the nodes


Also in this case, the mined block will not contain not allowed transaction as in this case

# Miner Wallet

Every Miner has a wallet in which the fees of the transactions are charged. Let's inspect the wallets of the miners at this point

In [42]:
print(miner1.wallet, miner2.wallet, miner3.wallet, miner4.wallet)

0 0.2 0.5 0


# Reading Permission

The last feature that the blockchain implements is a temporary couple of private and public keys for each patient, so that the doctor can access the history of the patient only one time without future access to the patient's data 

In [43]:
# Let's first store the orivate key fo the patient
private_key = patient3.private_key

# Now let's access the history of the patient using the private key
print(Health_block.get_patient_history(patient3, private_key=private_key))

Temporary keys have been refreshed. Old keys have been destroyed and will not work.
['illness1']


In [44]:
# If now we attempt to reaccess the history again, this will not be possible
print(Health_block.get_patient_history(patient3, private_key=private_key))

PermissionError: You do not have access to the BlockChain of this patient.